In [154]:
import pandas as pd 
import numpy as np 

Purpose:
- Clean raw transactional data
- Handle missing values, outliers, duplicates
- Standardize fields
- Enrich with holiday information

Output:
- online_retail_cleaned.csv


# Data Extraction / Loading

In [155]:
raw_online_retail=pd.read_csv("../data/raw/online_retail.csv")
raw_public_holidays = pd.read_csv("../data/external/publicHolidays.csv")
print(raw_online_retail.columns)
print(raw_public_holidays.columns)

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')
Index(['Unnamed: 0', 'countryOrRegion', 'holidayName', 'normalizeHolidayName',
       'isPaidTimeOff', 'countryRegionCode', 'date'],
      dtype='object')


#  Data Cleaning & Preprocessing

## 1 Cleaning Online Retail Dataset

In [156]:
clean_online_retail = raw_online_retail.copy()
clean_public_holidays = raw_public_holidays.copy()

Customer Purchasing Behavior

Goal: Understand repeat purchases, purchase frequency, and customer spend.  

- Key Steps:

  - Removed rows with missing `CustomerID`.  

  - Removed purchases with `Quantity <= 0` or `UnitPrice <= 0`.  

  - Converted `InvoiceDate` to datetime.  

- Outcome: Dataset ready to calculate metrics like total spend per customer, average purchase frequency, and repeat purchase rate.


In [157]:
clean_online_retail['CustomerID'].isnull().sum()


np.int64(135080)

In [158]:
clean_online_retail=clean_online_retail.dropna(subset=['CustomerID'])
clean_online_retail['CustomerID'].isnull().sum()

np.int64(0)

b.check for duplicate records aka purchases with InvoiceNo
* StockCode
* Quantity
* UnitPrice 
* remove duplicate records

In [159]:
clean_online_retail.duplicated(subset=['InvoiceNo','StockCode','Quantity','UnitPrice'])
clean_online_retail=clean_online_retail.drop_duplicates(subset=['InvoiceNo','StockCode','Quantity','UnitPrice'])


split the order record data into 3 different datasets 
* purchased_dataset ->Quantity>0
* invalid_purchase_dataset= Quantity==0
* return_dataset ->Quantity<0

In [160]:
purchased_dataset =clean_online_retail[clean_online_retail['Quantity']>0].copy()
invalid_purchase_dataset=clean_online_retail[clean_online_retail['Quantity']==0].copy()
return_dataset=clean_online_retail[clean_online_retail['Quantity']<0].copy()

In [161]:
clean_online_retail.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

DATA CLEANING IN PURCHASED_DATASET

2.Convert features to appropriate datatypes


In [162]:
purchased_dataset['InvoiceDate']=pd.to_datetime(purchased_dataset['InvoiceDate'])

Product Performance
- Goal: Identify top-selling and revenue-generating products.  

- Key Steps:
  - Ensured all `StockCode` values exist. 

  - Standardized `Description` (stripped whitespace).  

  - Quantity: Extreme high quantities capped at upperbound_q

UnitPrice: Rows with zero, negative, or extremely high prices removed (or filtered)

- Outcome: Prepared data for Top-N product analysis and product-level trend insights.

In [163]:

purchased_dataset=purchased_dataset[purchased_dataset['StockCode'].notnull()]



2.standardizing categorical/text data.
 * strip edges
 * convert to lowercase
 * remove special charecters

In [164]:
purchased_dataset_stand_cat=purchased_dataset.copy()
purchased_dataset_stand_cat['Description']=purchased_dataset_stand_cat['Description'].str.strip().str.lower().str.replace(r'[^a-z0-9 ]', '', regex=True)


3.Identify outliers of Quantity

Quantity Outliers(Only Quanity is checked because In a typical retail/customer purchasing dataset like the one you’re working on, Quantity is often the main numeric column that can have extreme or suspicious values.)

**Definition:**  
Quantity outliers are unusually large or small order quantities that deviate significantly from the typical purchase behavior. These extreme values can skew key metrics like average sales per order.

I**mpact on Analysis:**  
- Extremely large orders can inflate averages and give a misleading view of customer behavior.  
- Extremely small or negative quantities (e.g., returns) can distort insights.



### Detect outlier is Quantity using IQR method

In [165]:

Q1_q=purchased_dataset_stand_cat['Quantity'].quantile(0.25)
Q3_q=purchased_dataset_stand_cat['Quantity'].quantile(0.75)
IQR_q=Q3_q-Q1_q

upperbound_q=Q3_q+1.5*IQR_q

# vals >lb=Q1-1.5*1qr and < Q3+1.5*1qr
quantity_outliers=purchased_dataset_stand_cat[(purchased_dataset_stand_cat['Quantity']>upperbound_q)]
print(quantity_outliers)

       InvoiceNo StockCode                       Description  Quantity  \
9         536367     84879     assorted colour bird ornament        32   
31        536370     10002        inflatable political globe        48   
44        536370     22492            mini paint set vintage        36   
46        536371     22086     paper chain kit 50s christmas        80   
65        536374     21258        victorian sewing box large        32   
...          ...       ...                               ...       ...   
541835    581579     23581            jumbo bag paisley park        40   
541865    581583     20725           lunch bag red retrospot        40   
541866    581583     85038    6 chocolate love heart tlights        36   
541867    581584     20832  red flock love heart photo frame        72   
541868    581584     85038    6 chocolate love heart tlights        48   

               InvoiceDate  UnitPrice  CustomerID         Country  
9      2010-12-01 08:34:00       1.69     1

### Handling Outliers in Quantity


Extreme High Quantities (Bulk or Mistakes):

Extremely high quantities can skew metrics like total spent, frequency, and repeat purchases. Cap them using the IQR method to keep the analysis realistic.

In [166]:
purchased_df = purchased_dataset_stand_cat.copy()
purchased_df['Quantity'] = purchased_df['Quantity'].clip(
    upper=upperbound_q
)

In [167]:
purchased_df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

Detecting Outliers in Unitprice


In [168]:
Q1_u=purchased_df['UnitPrice'].quantile(0.25)
Q3_u=purchased_df['UnitPrice'].quantile(0.75)
IQR_u=Q3_u-Q1_u
lowerbound_u=Q1_u-1.5*IQR_u
upperbound_u=Q3_u+1.5*IQR_u
Unitprice_outliers_lowerbound=purchased_df[purchased_df['UnitPrice']>lowerbound_u]
Unitprice_outliers_upperbound=purchased_df[purchased_df['UnitPrice']<upperbound_u]



Handling Outliers in Unitprice


In [169]:
#negative outliers and extream large positive outliers of UnitPrice
new_purchased_dataset=purchased_df.copy()
new_purchased_dataset=purchased_df[(purchased_df['UnitPrice']>0)&(purchased_df['UnitPrice']<upperbound_u)].copy()


In [170]:
new_purchased_dataset.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

Revenue Drivers
- Goal: Determine main contributors to revenue.  
- Key Steps:  
  - Removed rows with invalid `Quantity` or `UnitPrice`.  

  - Created `Revenue = Quantity × UnitPrice`.  

  - Inspected revenue distribution for extreme values.  
  
- **Outcome:** Ready to identify top customers, top products, and main revenue contributors.


In [171]:
new_purchased_dataset['Revenue']=new_purchased_dataset['Quantity']*new_purchased_dataset['UnitPrice']

In [172]:
new_purchased_dataset['Revenue'].describe()

count    357993.000000
mean         14.600212
std          16.364582
min           0.001000
25%           4.200000
50%          10.500000
75%          17.700000
max         201.150000
Name: Revenue, dtype: float64

In [173]:
neg_revenue = new_purchased_dataset[new_purchased_dataset['Revenue'] < 0]
print(neg_revenue.shape)
print(neg_revenue.head())
zero_revenue = new_purchased_dataset[new_purchased_dataset['Revenue'] == 0]
print(zero_revenue.shape)
print(zero_revenue.head())

(0, 9)
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country, Revenue]
Index: []
(0, 9)
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country, Revenue]
Index: []


Customer Retention Patterns
- Goal:Measure repeat purchase behavior and loyalty.  
- Key Steps:
  -  Already Dropped rows without `CustomerID`.  
  - Sorted by `CustomerID` and `InvoiceDate`.  
  - Extracted Year and Month to create cohorts.  
- Outcome: Enables calculation of repeat purchase rate, time between purchases, and customer lifetime value.


In [174]:
new_purchased_dataset_sorted=new_purchased_dataset.sort_values(by=['CustomerID','InvoiceDate'])

In [175]:
new_purchased_dataset_sorted['Year']=new_purchased_dataset_sorted['InvoiceDate'].dt.year
new_purchased_dataset_sorted['Month']=new_purchased_dataset_sorted['InvoiceDate'].dt.month
new_purchased_dataset_sorted['day_of_week'] = new_purchased_dataset_sorted['InvoiceDate'].dt.dayofweek
new_purchased_dataset_sorted['week'] = new_purchased_dataset_sorted['InvoiceDate'].dt.isocalendar().week


#Find the first purchase date per customer
new_purchased_dataset_sorted['CohortMonth']=new_purchased_dataset_sorted.groupby('CustomerID')['InvoiceDate'].transform('min').dt.to_period('M')


In [176]:
clean_online_retail=new_purchased_dataset_sorted.copy()

## 2 External Holiday Dataset 

In [177]:
clean_public_holidays.columns=(clean_public_holidays.columns.str.lower().str.strip().str.replace(" ","_"))

In [178]:
clean_public_holidays.columns

Index(['unnamed:_0', 'countryorregion', 'holidayname', 'normalizeholidayname',
       'ispaidtimeoff', 'countryregioncode', 'date'],
      dtype='object')

In [179]:
clean_public_holidays=clean_public_holidays[['date','countryorregion','holidayname']]

In [180]:
req_public_holidays=clean_public_holidays.copy()
req_public_holidays['date']=pd.to_datetime(req_public_holidays['date'])

req_public_holidays=req_public_holidays.dropna(subset=['date','countryorregion'])
req_public_holidays=req_public_holidays.drop_duplicates(subset=['date','countryorregion','holidayname'])
req_public_holidays['day_of_week']=req_public_holidays['date'].dt.weekday
req_public_holidays['month']=req_public_holidays['date'].dt.month
req_public_holidays['is_weekend']=req_public_holidays['date'].dt.weekday>5
req_public_holidays['month_end']=req_public_holidays['date'].dt.is_month_end
req_public_holidays['month_start']=req_public_holidays['date'].dt.is_month_start

In [181]:
req_public_holidays['countryorregion'] = req_public_holidays['countryorregion'].astype(str).str.strip().str.title()
categorical_features = ['countryorregion']



In [182]:
req_public_holidays['holidayname']=req_public_holidays['holidayname'].str.strip().str.title()
req_public_holidays['isholiday']=(req_public_holidays['holidayname'].notna().astype(int))

In [183]:
req_public_holidays.columns
req_public_holidays.to_csv('../data/cleaned/holiday_dataset.csv',index=False)
clean_online_retail.to_csv('../data/cleaned/online_retail_dataset.csv',index=False)


#  Merging External Data into Cleaned Dataset

In [184]:

clean_online_retail=pd.merge(clean_online_retail,req_public_holidays,left_on=['InvoiceDate','Country'],right_on=['date','countryorregion'],how='left')


In [185]:
print(clean_online_retail)

       InvoiceNo StockCode                          Description  Quantity  \
0         541431     23166       medium ceramic top storage jar        27   
1         537626     85116       black candelabra tlight holder        12   
2         537626     22375    airline bag vintage jet set brown         4   
3         537626     71477      colour glass star tlight holder        12   
4         537626     22492               mini paint set vintage        27   
...          ...       ...                                  ...       ...   
357988    570715     22419                     lipstick pen red        12   
357989    570715     22866        hand warmer scotty dog design        12   
357990    573167     23264   set of 3 wooden sleigh decorations        27   
357991    573167     21824  painted metal star with holly bells        27   
357992    573167     21014         swiss chalet tree decoration        24   

               InvoiceDate  UnitPrice  CustomerID         Country  Revenue 

In [186]:
clean_online_retail= clean_online_retail.drop(
    columns=['day_of_week_y']
)

clean_online_retail = clean_online_retail.rename(
    columns={'day_of_week_x': 'day_of_week'}
)


In [187]:
clean_online_retail.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Revenue', 'Year', 'Month',
       'day_of_week', 'week', 'CohortMonth', 'date', 'countryorregion',
       'holidayname', 'month', 'is_weekend', 'month_end', 'month_start',
       'isholiday'],
      dtype='object')

In [188]:
clean_online_retail=clean_online_retail.dropna(subset=['InvoiceNo',
    'StockCode',
    'Quantity',
    'UnitPrice',
    'InvoiceDate',
    'Revenue'])

In [189]:
clean_online_retail['CustomerID'].isnull().sum()

np.int64(0)

In [190]:
clean_online_retail['isholiday'].isnull().sum()

np.int64(357993)

In [191]:
clean_online_retail['isholiday']=(clean_online_retail['isholiday'].fillna(False) .infer_objects(copy=False))

C:\Users\meena\AppData\Local\Temp\ipykernel_5132\2470773520.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clean_online_retail['isholiday']=(clean_online_retail['isholiday'].fillna(False) .infer_objects(copy=False))


In [192]:
clean_online_retail['holidayname'].isnull().sum()

np.int64(357993)

In [193]:
clean_online_retail['holidayname']=clean_online_retail['holidayname'].fillna("Non-holiday")

In [194]:
clean_online_retail['HolidayQuantity']=(clean_online_retail['Quantity']*clean_online_retail['isholiday'])

In [195]:
clean_online_retail['HolidayQuantity'].isnull().sum()

np.int64(0)

In [196]:
clean_online_retail.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Revenue', 'Year', 'Month',
       'day_of_week', 'week', 'CohortMonth', 'date', 'countryorregion',
       'holidayname', 'month', 'is_weekend', 'month_end', 'month_start',
       'isholiday', 'HolidayQuantity'],
      dtype='object')

In [197]:
clean_online_retail[['Month', 'month']].drop_duplicates()

,Month,month
0,1,NaN
1,12,NaN
60,4,NaN
83,6,NaN
100,8,NaN
120,10,NaN
201,9,NaN
203,11,NaN
262,2,NaN
291,3,NaN


In [198]:
clean_online_retail.drop(columns=['month'])
clean_online_retail.rename(columns={'Month':'Month_num'},inplace=True)

In [199]:
clean_online_retail.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Revenue', 'Year', 'Month_num',
       'day_of_week', 'week', 'CohortMonth', 'date', 'countryorregion',
       'holidayname', 'month', 'is_weekend', 'month_end', 'month_start',
       'isholiday', 'HolidayQuantity'],
      dtype='object')

In [200]:
clean_online_retail['YearMonth']=clean_online_retail['InvoiceDate'].dt.to_period('M')
clean_online_retail.to_csv("../data/cleaned/online_retail_cleaned.csv",index=False)
clean_online_retail.to_csv("online_retail_clean_backup.csv",index=False)